# 1D difussion equation solver implementation

In this notebook, we provide a pedagogical implementation of the 1D diffusion equation solver via the finite volume method. We will use our reasoning below to eventually devise the final prototype code.

The general equation to solve is the following:

$$
\frac{\partial \phi}{\partial t} = \frac{\partial }{\partial x}\left(\Gamma \frac{\partial \phi}{\partial x}\right) + S_\phi; \hspace{5mm} \phi = \phi(x, t), \, \Gamma = \Gamma(x, t).
$$
In the inner term on the right-hand-side, we recognise the flux term, $J = -\Gamma \frac{\partial \phi}{\partial x}$ with $Gamma$ being the corresponding transport coefficient while $S_\phi$ denote the source terms. To solve the above equation, we first discretize the interval $[x_0, x_0 + L]$ into $N$ equal
parts $x_{i, i+1}$ of length $\Delta x = L/N$. To apply the [finite volume method](https://en.wikipedia.org/wiki/Finite_volume_method), take into account the conservation laws (the continuity equation) present in our problem by integrating the original equation over the volume of each subinterval. After making use of the divergence theorem, we obtain:
$$
\frac{\partial \overline{\phi_i}}{\partial t} \Delta x = J(e_i) - J(w_i) + \overline{S_i}\Delta x.
$$
This is nothing more than a continuity equation for the $i$-th subinterval, stating that the rate of change of $\overline{\phi_i}$ (averaged over a given interval) is proportional to the difference of fluxes passing through the interval boundaries along with the possible contributions of the source/drain terms (again, the overline denotes averaging over the chosen subinterval). The numerical details now enter through the appropriate choice of the discretization scheme used to represent the fluxes along with the choice of a given time propagator to advance the solutions in time. We also need to take care of the boundary and initial conditions, which we will do below as we go through the process. 

Note: as standard in the literature, we used $J(w_i)$ and $J_(e_i)$ do denote the fluxes at the western (left) and eastern (right) cell boundary, respectivelly. It is important to remember that in the FVM, the values of the discretized quantities on the grid refer to the cell center values, while the fluxes (and corresponding transport coefficients) are evaluated at the cell faces. This, particularly in higher dimensions, importantly distinguishes FMV from the finite difference method (FDM). The most important difference already evident in 1D pertains to the treatment of boundary conditions. 

In [1]:
# import the relevant libraries
import numpy as np
import numba as nb


## Define grid

The problem is solved on a uniform grid of $N$ points on an interval $[x0, x0 + L].$

In [2]:
x0 = 0.
L = 1.

N = 10

mesh = np.linspace(x0, L, N)

delta_x = mesh[1] - mesh[0]

## Representing fluxes

To represent the flux through the eastern face of the $i$-th block, we have: 
$J(e_i) = \Gamma(e_i) \frac{\partial \phi}{\partial x}\rvert_{e_i}$ (and analogously for the western face). We now need to 
express the values of the derivatives at the cell centers using the known values of $\phi_i$ at the cell centers. To do so, we make
use of the Taylor expansions, which will eventually enable us to express the required derivatives in terms of the known quantities:
$$
\phi_{i+1} = \phi_e + \frac{\Delta x}{2}\frac{{\rm d} \phi}{{\rm d}x}\Big\rvert_e + \frac{1}{2}\left(\frac{\Delta x}{2}\right)^2\frac{{\rm d}^2\phi}{{\rm d} x^2}\Big\rvert_e + \frac{1}{6}\left(\frac{\Delta x}{2}\right)^3\frac{{\rm d}^3\phi}{{\rm d} x^3}\Big\rvert_e + \ldots
$$
$$
\phi_{i} = \phi_e - \frac{\Delta x}{2}\frac{{\rm d} \phi}{{\rm d}x}\Big\rvert_e + \frac{1}{2}\left(\frac{\Delta x}{2}\right)^2\frac{{\rm d}^2\phi}{{\rm d} x^2}\Big\rvert_e - \frac{1}{6}\left(\frac{\Delta x}{2}\right)^3\frac{{\rm d}^3\phi}{{\rm d} x^3}\Big\rvert_e + \ldots
$$
Subtracting the above equations, we get:
$$
\frac{{\rm d} \phi}{{\rm d}x}\Big\rvert_e = \frac{\phi_{i+1} - \phi_i}{\Delta x} - \frac{(\Delta x)^2}{24}\frac{{\rm d}^3 \phi}{{\rm d}x^3} + \ldots
$$
We see that the truncation error is second order in $\Delta x$. Also, due to the boundary conditions, the above scheme is valid for $i=2, \ldots, N-1,$ but not at the boundary cells. We thus have:

$$
\frac{{\rm d} \phi_i}{{\rm d} t}=
\Gamma_{e_i} \frac{\phi_{i+1} - \phi_i}{\Delta x} - \Gamma_{w_i} \frac{\phi_{i} - \phi_{i-1}}{\Delta x} + S_i \Delta x, \hspace{5mm} \forall i=2, \ldots, N-1.
$$
We will return to the treatment of the boundary conditions in a short while. For now, let us comment a bit on the values of the transport coefficients. These, too, are evaluated at the
cell faces but we only know the values of quantities at the cell centers (in principle, transport coefficients can be functions of $\phi$ and its derivatives). One of the simplest possible approaches
is to evaluate the face values from arithmetic means of the neighbouring cell-centered values:
$$
\Gamma_{e_i} = \frac{\Gamma_i + \Gamma_{i+1}}{\Delta x}, \hspace{5mm} \Gamma_{w_i} = \frac{\Gamma_{i-1} + \Gamma_{i}}{\Delta x}.
$$
This approach can yield wrong results in case of discontinuities. A vanishing transport coefficient in one of the cells, for instance, should also imply vanishing transport in that region; however, as per above equation, the face-centered transport coefficient would still be finite. To avoid this, we can use a harmonic mean instead; for a uniform grid, the expression reads:
$$
\Gamma_{e_i} = \frac{2(\Gamma_i  \Gamma_{i+1})}{\Gamma_i + \Gamma_{i+1}}.
$$
This yields the expected expression in case of a uniform transport coefficient and vanishes if any of the coefficients is zero.

For the boundary cell, we use a second-order approximation for the transport coefficient:

$$
\Gamma_{w_1} = \Gamma_L \approx \frac{3\Gamma_1 - \Gamma_2}{2}
$$

### Writing down the extrapolation schemes for face values

In [3]:
@nb.njit()
def _face_coeffs(coeffs_cell, coeffs_face):
    """
    Return an array of
    transport coefficients
    evaluated at cell faces.
    Use the harmonic interpolation
    scheme.

    Parameters:
    -----------
    coeffs_cell: ndarray, 1D
                Values of the cell-centered transport
                coefficients.
    
    coeffs_face: ndarray, 1D
                Values of the face-centered transport coefficients.
                The size of the array should be greater than the
                size of the coeffs_cell array by one element.

    Returns:
    --------

    coeffs_face: ndarray, 1D
                Values of the face-centered transport coefficients.

    """

    coeffs_face[1:-1] = 2*coeffs_cell[:-1] * \
        coeffs_cell[1:]/(coeffs_cell[:-1]+coeffs_cell[1:])

    coeffs_face[0] = (3 * coeffs_cell[0] - coeffs_cell[1]) / 2.
    coeffs_face[-1] = (3 * coeffs_cell[-1] - coeffs_cell[-2]) / 2.
    
    return coeffs_face

@nb.njit()
def _face_vals(vals_cell, coeffs_cell, vals_face):
    """
    Return values of the dependent variable at cell
    faces. We use the transport-coefficient-weighted
    approach that can properly account for
    discontinuities as well. 

    Parameters:
    -----------
    vals_cell: ndarray, 1D
                Values evaluated at the cell centers.
    
    coeffs_face: ndarray, 1D
                Values evaluated at faces.
                The size of the array should be greater than the
                size of the coeffs_cell array by one element.

    Returns:
    --------

    vals_face: ndarray, 1D
                Values evaluated at the faces
                in the interior of the interval, the first and
                the last value depend on the chosen boundary conditions.

    """
    term1 = vals_cell[1:] * coeffs_cell[1:]
    term2 = vals_cell[:-1] * coeffs_cell[:-1]
    term3 = coeffs_cell[1:] + coeffs_cell[:-1]

    vals_face[1:-1] = (term1 + term2) / term3
    
    return vals_face




### Application of the boundary conditions

#### Dirichlet case

In the Dirichlet case, we fix the independent variable at the boundary, $\phi_{w_1} =  \phi_L$. This brings about the condition for the flux at the boundary. To obtain
a second-order expression, we perform two Taylor expansions:

$$
\phi_{1} = \phi_{L} + \frac{\Delta x}{2}\frac{{\rm d} \phi}{{\rm d}x}\Big\rvert_{L} + \frac{1}{2}\left(\frac{\Delta x}{2}\right)^2\frac{{\rm d}^2\phi}{{\rm d} x^2}\Big\rvert_{L} + \ldots
$$

$$
\phi_{2} = \phi_{L} + \frac{3\Delta x}{2}\frac{{\rm d} \phi}{{\rm d}x}\Big\rvert_{L} + \frac{1}{2}\left(\frac{3\Delta x}{2}\right)^2\frac{{\rm d}^2\phi}{{\rm d} x^2}\Big\rvert_{L} + \ldots
$$

After multiplying the first equation by 9, subtracting the two and rearranging, we obtain:

$$
\frac{{\rm d} \phi}{{\rm d} x }\Big\rvert_{L} = \frac{9\phi_1 -  \phi_2 - 8\phi_L}{3\Delta x}.
$$

Equation for the boundary cell then reads:

$$
\frac{{\rm d} \phi_1}{{\rm d} t}=
\frac{2\Gamma_1\Gamma_2}{\Gamma_1 + \Gamma_2} \frac{\phi_{2} - \phi_1}{\Delta x} - \frac{3\Gamma_1 - \Gamma_2}{2} \frac{9\phi_1 -  \phi_2 - 8\phi_L}{3\Delta x} + S_i \Delta x,
$$

#### Von Neumann case

In the von Neumann case, we fix the value of the flux at the boundary, consequently, we need to determine the boundary temperature using some postprocessing step.

$$
\frac{{\rm d} \phi}{{\rm d} x}\Big\rvert_{L} = \frac{{\rm d} \phi}{{\rm d} x}\Big\rvert_{w_1} = J_L
$$

At the boundary cell, we now have:

$$
\frac{{\rm d} \phi_1}{{\rm d} t}=
\frac{2\Gamma_1\Gamma_2}{\Gamma_1 + \Gamma_2} \frac{\phi_{2} - \phi_1}{\Delta x} - \frac{3\Gamma_1 - \Gamma_2}{2} J_L + S_i \Delta x,
$$

### Putting it all together

The general (interior) term is written as:

$$
\frac{{\rm d} \phi_i}{{\rm d} t}= -\phi_i \frac{\Gamma_{e_i} + \Gamma_{w_i}}{\Delta x} + 
\phi_{i+1} \frac{\Gamma_{e_i}}{\Delta x} +\phi_{i-1} \frac{\Gamma_{w_i}}{\Delta x} + S_i \Delta x, \hspace{5mm} \forall i=2, \ldots, N-1.
$$


In [15]:
@nb.njit()
def _discretization(coeffs_face, delta_x, bc1=0, bc2=0):
    
    N = np.shape(coeffs_face)[0] - 1
    diag = np.zeros(N)
    subdiag = np.zeros(N - 1)
    superdiag = np.zeros_like(subdiag)

    for i in range(1, N-1):

        diag[i] = - (coeffs_face[i] + coeffs_face[i+1]) / delta_x
        subdiag[i-1] = coeffs_face[i] / delta_x
        superdiag[i] = coeffs_face[i+1] / delta_x
    
    # boundary conditions

    # left edge
    if bc1 == 0:

        diag[0] = -(coeffs_face[1] + coeffs_face[0]*3.) / delta_x
        superdiag[0] = (coeffs_face[1] + coeffs_face[0]/3.) / delta_x
    
    elif bc1 == 1:

        diag[0] = - coeffs_face[1] / delta_x
        superdiag[0] = coeffs_face[1] / delta_x
    
    # right edge
    if bc2 == 0:

        diag[-1] = -(coeffs_face[-2] + coeffs_face[-1]*3) / delta_x
        subdiag[-1] = (coeffs_face[-2] + coeffs_face[-1]/3.) / delta_x
    
    elif bc2 == 1:

        diag[-1] = - coeffs_face[-2] / delta_x
        subdiag[-1] = coeffs_face[-2] / delta_x
    
    return diag, subdiag, superdiag



    

In [16]:
# uniform coeffs
coeffs_cell = np.ones_like(mesh)
coeffs_face = np.zeros(coeffs_cell.shape[0] + 1)
coeffs_face = _face_coeffs(coeffs_cell, coeffs_face)

phi0 = np.zeros_like(mesh)
phi0[3] = 1
phi_face = np.zeros_like(coeffs_face)
phi_cell = _face_vals(phi0, coeffs_cell, phi_face)

diag, subdiag, superdiag = _discretization(coeffs_face, delta_x, 0, 0 )

In [19]:
from numba.experimental import jitclass
from numba import int32, float64
class control_volume(object):
    def __init__(self, value):
        self.value = value
        self.array = np.zeros(value, dtype=np.float64)

4